In [2]:
%load_ext autoreload
%autoreload 2

import joblib
import warnings
import tensorflow as tf

from src.models import LeNet
from src.data_utils import *

warnings.filterwarnings("ignore")
tf.get_logger().setLevel(tf.logging.ERROR)
np.random.seed(42)

# Train LeNet

In [26]:
(X_train, y_train), (X_test, y_test) = get_mnist()
X_train, X_train_db, y_train, y_train_db = split_to_create_db(X_train, y_train, fold_size=0.2)

In [7]:
# prepare the model
lenet = LeNet(
    input_shape=X_train.shape[1:],
    num_classes=10,
)

epochs = 20
mc_rate = 0.5
lenet.set_mc_dropout_rate(mc_rate)
lenet.train(X_train, y_train, X_test, y_test, epochs=epochs, verbose=1)

# lenet.load_model("Assets/lenet-0.5-20-4folds.h5")

Training with mc_dropout_rate = 0.5.

Train on 48000 samples, validate on 10000 samples
Epoch 1/20
48000/48000 [==============================] - 4s 90us/step - loss: 0.1767 - acc: 0.9456 - val_loss: 0.0772 - val_acc: 0.9753
Epoch 2/20
48000/48000 [==============================] - 5s 98us/step - loss: 0.0709 - acc: 0.9779 - val_loss: 0.0541 - val_acc: 0.9826
Epoch 3/20
48000/48000 [==============================] - 6s 122us/step - loss: 0.0585 - acc: 0.9812 - val_loss: 0.0560 - val_acc: 0.9821
Epoch 4/20
48000/48000 [==============================] - 5s 114us/step - loss: 0.0483 - acc: 0.9848 - val_loss: 0.0616 - val_acc: 0.9827
Epoch 5/20
48000/48000 [==============================] - 5s 98us/step - loss: 0.0434 - acc: 0.9869 - val_loss: 0.0483 - val_acc: 0.9866
Epoch 6/20
48000/48000 [==============================] - 5s 106us/step - loss: 0.0394 - acc: 0.9885 - val_loss: 0.0566 - val_acc: 0.9837
Epoch 7/20
48000/48000 [==============================] - 6s 131us/step - loss: 0.0358 

In [8]:
lenet.save_model(f'Assets/lenet-{mc_rate}-{epochs}-4folds')

In [9]:
def mc_dropout(net, X_train, batch_size=1000, dropout=0.5, T=100):
    """
    net: keras model with set_mc_dropout_rate function

    Forward passes T times, then take the variance from all the predictions for each class.
    the mc_dropout score for an example will be the mean of the variances for all the classes.
    y_mc_dropout is the mean of all runs.
    """
    net.set_mc_dropout_rate(dropout)
    model = net.model
    repetitions = []
    # Todo: parallelize
    for _ in range(T):
        pred = model.predict(X_train, batch_size)
        repetitions.append(pred)

    net.set_mc_dropout_rate(0)
    preds = np.array(repetitions)  # T x data x pred

    # average over all passes
    y_mc_dropout = preds.mean(axis=0)

    # get variance from all preds for each example (output: batch x preds classes) each cell is var
    mc = np.var(preds, axis=0)
    # mean of vars of each class (out: one dim array with batch as dim)
    mc_uncertainty = np.mean(mc, axis=-1)

    return y_mc_dropout, -mc_uncertainty

def create_db(net, X_train_db, max_mc_iters):
    from tqdm import tqdm
    db = {}
    for t in tqdm(range(2, max_mc_iters+1)):
        mean, var = mc_dropout(net, X_train_db, T=t)
        db[t] = (mean, var)    
    joblib.dump(db, f"Assets/db_{max_mc_iters}_iters.jblib", compress=True)
    print(f"db_{max_mc_iters}_iters created")

In [10]:
ans = create_db(lenet, X_train_db, 1000)

100%|██████████| 999/999 [8:32:45<00:00, 30.80s/it]  


db_1000_iters created


In [28]:
%%time
db = joblib.load("Assets/db_1000_iters.jblib")

Wall time: 2.99 s
